<a href="https://colab.research.google.com/github/davidbejkovski/DIANS_2024/blob/master/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B0%201/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML
import os
from datetime import datetime, timedelta

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date

In [ ]:
requests.packages.urllib3.disable_warnings()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
url="https://www.mse.mk/mk/issuers/free-market"
base_url="https://www.mse.mk"
answer = requests.get(url)

In [ ]:
from selenium.webdriver.support.ui import Select

In [ ]:
def Filter2(url):
  request = requests.get(url)
  text=request.text
  soup=BeautifulSoup(text,'html.parser')
  body = soup.select('.col-md-12 a')
  links = [a['href'] for a in body if a.has_attr('href')]
  return links[0]

In [ ]:
def get_last_date_csv(csv_file):
    filename = f'{csv_file}.csv'
    if os.path.exists(filename):
        df = pd.read_csv('/content/'+filename)
        if not df.empty:
            df['Датум'] = pd.to_datetime(df['Датум'], errors='coerce')
            last_date = df['Датум'].max()
            if pd.notna(last_date):
                return last_date.strftime("%d.%m.%Y")
        else:
            return None
    else:
        columns = ['Датум', 'Цена на последна трансакција', 'Мак.', 'Мин.', 'Просечна цена', '%пром.', 'Количина', 'Промет во БЕСТ во денари', 'Вкупен промет во денари']
        df = pd.DataFrame(columns=columns)
        df.to_csv(filename, index=False)

In [ ]:
def get_data(csv_file, url, start_date, end_date):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    data = []
    start_date_picker=driver.find_element(By.ID,'FromDate')
    start_date_picker.clear()
    start_date_picker.send_keys(start_date)
    end_date_picker=driver.find_element(By.ID,'ToDate')
    end_date_picker.clear()
    end_date_picker.send_keys(end_date)
    button=driver.find_element(By.CLASS_NAME,'btn-primary-sm')
    driver.execute_script('arguments[0].scrollIntoView();',button)
    button.click()
    time.sleep(1)
    request = driver.page_source
    soup = BeautifulSoup(request, 'html.parser')
    body = soup.select('tbody tr')
    for b in body:
      cols = b.find_all('td')
      if len(cols) >= 9:
          data.append({
                    "Датум": cols[0].text.strip(),
                    "Цена на последна трансакција": cols[1].text.strip(),
                    "Мак.": cols[2].text.strip(),
                    "Мин.": cols[3].text.strip(),
                    "Просечна цена": cols[4].text.strip(),
                    "%пром.": cols[5].text.strip(),
                    "Количина": cols[6].text.strip(),
                    "Промет во БЕСТ во денари": cols[7].text.strip()+',00',
                    "Вкупен промет во денари": cols[8].text.strip()+',00'
                })
    driver.quit()

    return data

In [11]:
text=answer.text
soup=BeautifulSoup(text,'html.parser')
body=soup.select("tbody tr a")
links = [a['href'] for a in body if a.has_attr('href')]
for i in range(0,len(links)):
  name=Filter2(base_url+links[i])
  if '#' in name:
    continue
  parts = name.split('/')
  name = parts[4]
  data = []
  if get_last_date_csv(name) is None:
      start_date=datetime(2014, 1, 1).strftime('%d.%m.%Y')
      end_date=datetime(2014,12,31).strftime('%d.%m.%Y')
      data+=get_data(name, base_url + Filter2(base_url + links[i]), start_date, end_date)
  else:
      start_date = get_last_date_csv(name)
      year_start = start_date.split('.')
      today = str(date.today())
      year_end = today.split('-')
      end_year = int(year_end[0])
      start_year = int(year_start[2])

      for year in range(start_year, end_year + 1):
          if year == end_year:
              end_month = int(year_end[1])
              end_day = int(year_end[2])
              end_date = datetime(year, end_month, end_day).strftime('%d.%m.%Y')
          else:
              end_date = datetime(year, 12, 31).strftime('%d.%m.%Y')
          start_date = datetime(year, 1, 1).strftime('%d.%m.%Y')
          data += get_data(name, base_url + Filter2(base_url + links[i]), start_date, end_date)
  df = pd.DataFrame(data)
  df.to_csv('/content/' + name + '.csv')



KeyboardInterrupt: 